In [1]:
import pandas as pd
import numpy as np

# read functions.
import os
for f in os.listdir('../fun/'): exec(open('../fun/'+f).read())
del f

# Load data
load( '../out/d3-fight-level-transform.pkl' )

X = pd.DataFrame(X)
X.columns = cols
X.shape

(4368, 165)

In [2]:
# find highly correlated pairs.
# for each, return the lowest-correlated one to drop later.
hicor = []
todrop = []
c = X.corr()
for row in range(c.shape[0]):
    for col in range(c.shape[1]):
        if col >= row: continue
        if c.iloc[row,col] > .8: 
            hicor.append( [ cols[row], cols[col] ] )
            rowcor = np.corrcoef(X.iloc[:,row],y)[0][1]
            colcor = np.corrcoef(X.iloc[:,col],y)[0][1]
            if rowcor < colcor:
                todrop.append( cols[row] )
            else:
                todrop.append( cols[col])
            
print('Correlated Pairs:')
print(hicor)
print('')
print('Dropping:')
print(todrop)

Correlated Pairs:
[['Mean_Height_cms', 'weight_class'], ['Mean_Reach_cms', 'weight_class'], ['Mean_Reach_cms', 'Mean_Height_cms'], ['Mean_total_rounds_fought', 'Mean_losses'], ['Diff_total_rounds_fought', 'Diff_losses'], ['Mean_win_by_Decision_Unanimous', 'Mean_total_rounds_fought'], ['Mean_wins', 'Mean_longest_win_streak'], ['Mean_wins', 'Mean_total_rounds_fought'], ['Mean_wins', 'Mean_win_by_Decision_Unanimous'], ['Diff_wins', 'Diff_total_rounds_fought']]

Dropping:
['Mean_Height_cms', 'Mean_Reach_cms', 'Mean_Reach_cms', 'Mean_losses', 'Diff_losses', 'Mean_total_rounds_fought', 'Mean_wins', 'Mean_total_rounds_fought', 'Mean_win_by_Decision_Unanimous', 'Diff_total_rounds_fought']


In [3]:
X.drop( todrop, axis = 1, inplace = True )

In [4]:
# Naive throw everything in
import statsmodels.api as sm
logit_model = sm.Logit( y, X )
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.611539
         Iterations 6
                                          Results: Logit
Model:                          Logit                       Pseudo R-squared:            0.035    
Dependent Variable:             Winner                      AIC:                         5650.4033
Date:                           2019-11-24 20:47            BIC:                         6633.2406
No. Observations:               4368                        Log-Likelihood:              -2671.2  
Df Model:                       153                         LL-Null:                     -2769.1  
Df Residuals:                   4214                        LLR p-value:                 0.011180 
Converged:                      1.0000                      Scale:                       1.0000   
No. Iterations:                 6.0000                                                            
-------------------------------------------------

c:\users\bryce\documents\python37\lib\site-packages\statsmodels\base\model.py:1286: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\bryce\documents\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [5]:
# drop variables below .1 p values.
todrop = list( result.pvalues[ result.pvalues > .1 ].index )
print( 'Dropping: ')
print(todrop)
X.drop( todrop, axis = 1, inplace = True )

Dropping: 
['title_bout', 'weight_class', 'no_of_rounds', 'date_dayofmonth', 'date_frisat', 'womens', 'weight_class_catch_weight', 'weight_class_open_weight', 'Diff_Height_cms', 'Mean_avg_KD', 'Diff_avg_KD', 'Mean_avg_PASS', 'Mean_avg_REV', 'Diff_avg_REV', 'Mean_avg_SUB_ATT', 'Diff_avg_SUB_ATT', 'Mean_avg_opp_SUB_ATT', 'Diff_avg_opp_SUB_ATT', 'Diff_avg_BODY_pct', 'Diff_avg_CLINCH_pct', 'Mean_avg_DISTANCE_pct', 'Diff_avg_DISTANCE_pct', 'Mean_avg_HEAD_pct', 'Diff_avg_HEAD_pct', 'Mean_avg_LEG_pct', 'Diff_avg_LEG_pct', 'Mean_avg_SIG_STR_pct', 'Diff_avg_SIG_STR_pct', 'Mean_avg_TD_pct', 'Diff_avg_TD_pct', 'Mean_avg_TOTAL_STR_pct', 'Mean_current_lose_streak', 'Diff_current_lose_streak', 'Mean_current_win_streak', 'Mean_longest_win_streak', 'Mean_total_time_fought(seconds)', 'Diff_total_time_fought(seconds)', 'Diff_total_title_bouts', 'Mean_win_by_Decision_Majority', 'Diff_win_by_Decision_Majority', 'Diff_win_by_Decision_Split', 'Diff_win_by_Decision_Unanimous', 'Mean_win_by_KO/TKO', 'Diff_win

In [6]:
# make predictions and check recall, precision, f1 score.

from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score

pred = result.predict()
print( 
    'Mean wins: %s \nMean predict: %s\n' % ( 
    y.mean(),
    pred.mean()
))

# what is our base level if we predict the majority?
print( 'Accuracy predicting all wins:\n')
print( classification_report( 
    y, 
    [ 1 for x in pred ]
))

# what is the outcome of different cutoffs?
print( 'Accuracy with varying cutoffs:\n' )
for i in range(11): 
    
    icutoff = i/10
    
    predwin = [ 1 if x > i/10 else 0 for x in pred ]
    predloss = [ 0 if x > i/10 else 1 for x in pred ]
    
    fscorewin = f1_score( y, predwin )
    fscoreloss = f1_score( ( y == 0 ) * 1, predloss )    
    prec = precision_score( y, predwin )
    recall = recall_score( y, predwin )
    
    print(
        '%s: \t f1-score: %s   \t precision %s   \t recall: %s' % ( 
            i/10, 
            round( (fscorewin + fscoreloss) / 2, 2 ),
            round( prec, 2 ),
            round( recall, 2 )
    ))

Mean wins: 0.6703296703296703 
Mean predict: 0.5105140339913664

Accuracy predicting all wins:

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1440
           1       0.67      1.00      0.80      2928

    accuracy                           0.67      4368
   macro avg       0.34      0.50      0.40      4368
weighted avg       0.45      0.67      0.54      4368

Accuracy with varying cutoffs:

0.0: 	 f1-score: 0.4   	 precision 0.67   	 recall: 1.0
0.1: 	 f1-score: 0.4   	 precision 0.67   	 recall: 1.0
0.2: 	 f1-score: 0.47   	 precision 0.68   	 recall: 0.99
0.3: 	 f1-score: 0.59   	 precision 0.72   	 recall: 0.91
0.4: 	 f1-score: 0.65   	 precision 0.77   	 recall: 0.77
0.5: 	 f1-score: 0.64   	 precision 0.83   	 recall: 0.59
0.6: 	 f1-score: 0.58   	 precision 0.89   	 recall: 0.43
0.7: 	 f1-score: 0.5   	 precision 0.95   	 recall: 0.28
0.8: 	 f1-score: 0.39   	 precision 0.98   	 recall: 0.15
0.9: 	 f1-score: 0.27   	 pr

c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\bryce\documents\python37\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


1.0: 	 f1-score: 0.25   	 precision 0.0   	 recall: 0.0


Seems like a cutoff of around .5 gives us way above average wins 
while participating in a large number of fights.
We are capturing 71% of the wins (recall) and winning 75% of the time (precision).
Strangely though, we could win 63% of the time and capture 100% of the wins by
always betting to win.
I guess we need to think about betting and what make the most sense.

In [7]:
# train-test split.
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state = 549,
    test_size = 0.25
)

# fit on train and get in-model accuracy.
m = sm.Logit( y_train, X_train ).fit()

Optimization terminated successfully.
         Current function value: 0.612652
         Iterations 8


In [8]:
print( 
    classification_report( 
        y_train, [ 1 if x > 0.5 else 0 for x in m.predict(X_train) ] 
))

              precision    recall  f1-score   support

           0       0.47      0.69      0.56      1068
           1       0.81      0.63      0.71      2208

    accuracy                           0.65      3276
   macro avg       0.64      0.66      0.63      3276
weighted avg       0.70      0.65      0.66      3276



In [9]:
print( 
    classification_report( 
        y_test, [ 1 if x > 0.5 else 0 for x in m.predict(X_test) ] 
))

              precision    recall  f1-score   support

           0       0.47      0.65      0.54       372
           1       0.77      0.62      0.69       720

    accuracy                           0.63      1092
   macro avg       0.62      0.63      0.61      1092
weighted avg       0.67      0.63      0.64      1092



In [10]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

from sklearn.model_selection import cross_validate
kfold = cross_validate( 
    model, X, y, 
    cv = 10, 
    scoring = [ 'precision', 'recall' ], 
    n_jobs = -1 
)
print( 
  'Mean precision: %s \nMean recall %s' % 
  ( kfold['test_precision'].mean(), kfold['test_recall'].mean() ) 
)

Mean precision: 0.689080056363391 
Mean recall 0.8126279863481228


Test accuracy is similar to in-model, so there doesn't seem to be overfitting. But then when we run k-fold we get much lower precision and higher recall, so there is over-fitting.